In [59]:
# part 1 (0) :
#==========================================================
# 📌 모듈 불러오기
#==========================================================

# 시스템
import os, sys, random, copy, glob, json
from typing import List, Dict, Optional

# EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 파이토치 (필요시)
import torch

# 유틸
from tqdm.auto import tqdm

# LangChain (LLM, 임베딩, 검색)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders import WikipediaLoader

# 기타
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
print('✅ 모듈 불러오기 - 완료')

✅ 모듈 불러오기 - 완료


In [60]:
# part 1 (1):
#==========================================================
# 📌 시드 고정(재현성)
#==========================================================
SEED = int(os.getenv("SEED", "2024"))
random.seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
try:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
except Exception:
    pass
print("✅ 시드 고정 - 완료")


✅ 시드 고정 - 완료


In [61]:
# part 1 (2) 
#===============================================================================
# ▶ 작업환경 플래그
#===============================================================================
IS_GOOGLE = True if 'google.colab'           in sys.modules   else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ    else False
IS_LOCAL  = True if not (IS_GOOGLE or IS_KAGGLE)              else False
print(f"✅ IS_GOOGLE={IS_GOOGLE}, IS_KAGGLE={IS_KAGGLE}, IS_LOCAL={IS_LOCAL}")


✅ IS_GOOGLE=False, IS_KAGGLE=False, IS_LOCAL=True


In [62]:
# part 1 (3) 
#==========================================================
# 📌 상수설정 (모델/토큰/메시지) + api_token.txt 로드
#==========================================================

# 🔑 API 키 (환경변수 권장) + 파일 경로
API_TOKEN_PATH  = os.getenv("API_TOKEN", "./api_token.txt")
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY", "")

# 파일에서 토큰 읽기 (환경변수 미설정 시만)
if not OPENAI_API_KEY and os.path.exists(API_TOKEN_PATH):
    try:
        with open(API_TOKEN_PATH, "r", encoding="utf-8") as f:
            _k = f.read().strip()
        if _k:
            os.environ["OPENAI_API_KEY"] = _k
            OPENAI_API_KEY = _k
    except Exception as e:
        print(f"⚠️ API 토큰 파일 읽기 실패: {e}")

if OPENAI_API_KEY:
    print("✅ OpenAI API 키 준비됨")
else:
    print("⚠️ OPENAI_API_KEY 미설정 — 실행 전 반드시 설정 또는 api_token.txt 준비")

# 🔧 LLM/임베딩 모델명
CHAT_MODEL_NAME  = os.getenv("OPENAI_CHAT_MODEL", "gpt-4o-mini")
EMBED_MODEL_NAME = os.getenv("OPENAI_EMBED_MODEL", "text-embedding-3-small")

# 🔢 토큰/파라미터
MAX_OUTPUT_TOKENS = int(os.getenv("MAX_OUTPUT_TOKENS", "800"))  # ✅ 요구: 토큰 상수화
TEMPERATURE       = float(os.getenv("TEMPERATURE", "0.2"))

# 🛡 도메인 제한 메시지
OFF_TOPIC_MESSAGE = "치매관련 상담만 가능합니다."

# 🧭 최초 고정 질문 4개
GUIDE_QUESTIONS = [
    "자주 쓰던 물건 이름이 갑자기 생각안 난적이 있나요?",
    "대화중단어가 잘 떠오르지 않아서 곤란했던 적이 있나요?",
    "가족이나 지인이 평소와 다르다고 한적이 있나요?",
    "최근에 불편했던 점이나 걱정되는 점이 있나요?"
]
print("✅ 상수설정 - 완료")


✅ OpenAI API 키 준비됨
✅ 상수설정 - 완료


In [63]:
# part 2 (1) 
#==========================================================
# 📌 페르소나, 템플릿, 정책
#==========================================================

SYSTEM_PERSONA = '''\
당신은 한국어로 상담하는 의학(치매진단) 상담 챗봇입니다.
- 역할: 환자/보호자의 서술(STT 텍스트)을 읽고, 지정된 템플릿으로 간결하고 체계적인 요약을 만듭니다.
- 태도: 정중하고 전문적이며, 과도한 확신/진단을 피하고 안전수칙을 강조합니다.
- 범위: 치매, 경도인지장애, 관련 증상/검사/일상 안전/가족 교육과 같은 주제에 한정합니다.
- 금지: 치매상담과 무관한 정보(예: 패스트푸드 신메뉴, 음료 당분 등)에 대한 응답 생성 금지.
- 출력: 반드시 아래 '요약 템플릿'을 정확한 형식으로 채워서 출력합니다.
'''

SUMMARY_TEMPLATE = '''\
<질문> 이부분은 고정4개중 1개 - 챗봇이 처음 제시한 가이드 질문
“{guide_question}”

<STT:시나리오:전체상담내용>
“{transcript}”

<요약>
1. **주 증상**
{symptoms_bullets}

2. **상담내용**
{counselling_bullets}

3. **심리상태**
{psych_bullets}

4. **AI 해석**
{ai_interp_bullets}

5. **주의사항**
{caution_bullets}
'''

from langchain_core.prompts import ChatPromptTemplate

SUMMARISE_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system",
       SYSTEM_PERSONA + "\n\n"
       "아래 '참고 문서(요약)'는 최신 지식 보강을 위한 참고용입니다. "
       "근거가 불명확하면 과도한 단정 대신 조심스러운 해석을 제시하세요.\n"
       "반드시 '요약 템플릿' 구조를 그대로 유지하십시오."
      ),
      ("human",
       "가이드 질문(고정 4개 중 선택): {guide_question}\n\n"
       "참고 문서(요약):\n{rag_context}\n\n"
       "사용자 STT 원문:\n{transcript}\n\n"
       "다음 요구사항을 따르세요:\n"
       "1) '주 증상'에는 핵심 증상만 '-' 불릿으로 간결히.\n"
       "2) '상담내용'에는 구체적 사건/진술 중심으로 불릿화.\n"
       "3) '심리상태'는 감정 단어(불안, 수치심, 무기력 등)를 사용.\n"
       "4) 'AI 해석'은 병명 단정 금지(예: '~가능성 시사').\n"
       "5) '주의사항'은 일상 안전, 정서적 지지, 검진 권고 등을 포함.\n"
       "6) 반드시 아래 형식으로 출력:\n"
       "{summary_template}\n"
       "출력 시 한국어 따옴표(“)를 유지하세요."
      )
    ]
)
print("✅ 페르소나/템플릿 - 준비 완료")


✅ 페르소나/템플릿 - 준비 완료


In [64]:
# part 2 (2) 
#==========================================================
# 📌 LLM/임베딩/웹검색(RAG) 구성 + Rerank-Then-Summarise (내결함성)
#  - OpenAI 키는 ./api_token.txt 또는 환경변수에서 로드되어 있어야 함
#  - duckduckgo-search 미설치 시 자동 설치 시도, 실패하면 Wikipedia 전용으로 동작
#  - chroma/vectorstore 실패시에도 동작하도록 fallback
#==========================================================

import sys, subprocess, numpy as np
from typing import List, Dict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document

# ---- OpenAI 키 점검 (앞선 part 1 (3)에서 설정되어 있어야 함) ----
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY and os.path.exists("./api_token.txt"):
    try:
        with open("./api_token.txt", "r", encoding="utf-8") as f:
            _k = f.read().strip()
        if _k:
            os.environ["OPENAI_API_KEY"] = _k
            OPENAI_API_KEY = _k
            print("✅ OpenAI API 키를 api_token.txt에서 로드했습니다.")
    except Exception as e:
        print(f"⚠️ API 토큰 파일 읽기 실패: {e}")

if not OPENAI_API_KEY:
    print("⚠️ OPENAI_API_KEY 미설정 — 실행 전 반드시 설정 또는 api_token.txt 준비")

# ---- LLM & Embeddings (명시적 api_key 전달) ----
llm = ChatOpenAI(
    model=CHAT_MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_OUTPUT_TOKENS,   # 일부 버전에선 max_completion_tokens
    api_key=OPENAI_API_KEY,
)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL_NAME, api_key=OPENAI_API_KEY)

# ---- DuckDuckGo 검색 래퍼 준비 (미설치 시 자동 설치 시도) ----
def _ensure_ddg_wrapper():
    try:
        from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
        return DuckDuckGoSearchAPIWrapper()
    except Exception as e:
        print(f"⚠️ DuckDuckGoSearchAPIWrapper 임포트 실패: {e}")
        # 자동 설치 시도
        try:
            print("↻ duckduckgo-search 설치 시도 중...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "duckduckgo-search"], stdout=subprocess.DEVNULL)
            from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
            print("✅ duckduckgo-search 설치 완료")
            return DuckDuckGoSearchAPIWrapper()
        except Exception as ee:
            print(f"⚠️ 설치 실패, Wikipedia 전용 fallback로 동작합니다: {ee}")
            return None

ddg = _ensure_ddg_wrapper()

# ---- Wikipedia 로더 & Chroma 준비 (실패시 fallback) ----
def _ensure_wikipedia_loader():
    try:
        from langchain_community.document_loaders import WikipediaLoader
        return WikipediaLoader
    except Exception as e:
        print(f"⚠️ WikipediaLoader 임포트 실패: {e}")
        return None

def _ensure_chroma():
    try:
        from langchain_community.vectorstores import Chroma
        return Chroma
    except Exception as e:
        print(f"⚠️ Chroma 임포트 실패(벡터스토어 없이 동작): {e}")
        return None

WikipediaLoader = _ensure_wikipedia_loader()
Chroma = _ensure_chroma()

# ---- 유틸 함수들 ----
def _to_docs_from_ddg_results(results: List[Dict]) -> List[Document]:
    docs = []
    for r in results:
        title   = r.get("title") or ""
        snippet = r.get("snippet") or r.get("body") or ""
        link    = r.get("link")   or r.get("href")  or ""
        text = f"{title}\n{snippet}\nURL: {link}"
        docs.append(Document(page_content=text, metadata={"source": link, "title": title}))
    return docs

def _embed_texts(texts: List[str]) -> List[List[float]]:
    return embeddings.embed_documents(texts)

def _embed_query(text: str) -> List[float]:
    return embeddings.embed_query(text)

def _cosine(a: List[float], b: List[float]) -> float:
    a = np.array(a, dtype=np.float32); b = np.array(b, dtype=np.float32)
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) + 1e-12
    return float(np.dot(a, b) / denom)

# ---- Rerank-Then-Summarise 컨텍스트 구축 ----
def build_rag_context_rerank(
    transcript: str,
    queries: List[str],
    k_search: int = 8,      # 검색 결과 개수
    k_vec: int    = 12,     # 벡터 스토어에서 1차 후보 개수
    k_rerank: int = 8,      # 재랭크 대상 개수
    k_final: int  = 4       # 최종 문맥으로 쓸 개수
) -> str:
    """
    1) DDG로 초기 검색(가능하면)
    2) (보조) 위키피디아
    3) 벡터스토어(Chroma) 1차 후보 추출
    4) '쿼리+트랜스크립트' 임베딩으로 코사인 유사도 재랭크
    5) 상위 k_final 문서를 문맥으로 결합
    """
    all_docs: List[Document] = []

    # 1) 검색
    if ddg is not None:
        for q in queries:
            try:
                # 질의 수에 따라 나눠서 검색 수 결정
                per_q = max(3, k_search // max(1, len(queries)) + 1)
                res = ddg.results(q, max_results=per_q)
                all_docs.extend(_to_docs_from_ddg_results(res))
            except Exception as e:
                print(f"⚠️ DuckDuckGo 검색 실패: {e}")
    else:
        print("ℹ️ DDG 미사용: Wikipedia만 사용합니다.")

    # 2) 위키피디아 보조 (없거나 적을 때)
    if (WikipediaLoader is not None) and (len(all_docs) < 2):
        try:
            wiki_docs = WikipediaLoader(query="치매", load_max_docs=3).load()
            all_docs.extend(wiki_docs)
        except Exception as e:
            print(f"⚠️ Wikipedia 로드 실패: {e}")

    if not all_docs:
        return ""

    # 3) 1차 후보 (Chroma → 실패 시 상위 k_vec 슬라이스)
    if Chroma is not None:
        try:
            vs = Chroma.from_documents(all_docs, embeddings)
            retriever = vs.as_retriever(search_kwargs={"k": min(k_vec, max(1, len(all_docs)))})
            seed_query = " ".join(queries[:3]) or "치매 경도인지장애 초기 증상 안전 가족 교육 평가도구"
            vec_docs = retriever.invoke(seed_query)
        except Exception as e:
            print(f"⚠️ 벡터스토어/리트리버 생성 실패: {e}")
            vec_docs = all_docs[:k_vec]
    else:
        vec_docs = all_docs[:k_vec]

    # 4) 재랭크 (cosine)
    combined_query = ((" ".join(queries)) + " " + transcript[:600]).strip()
    q_emb = _embed_query(combined_query)
    cand_docs  = vec_docs[:k_rerank]
    cand_texts = [d.page_content for d in cand_docs]
    cand_embs  = _embed_texts(cand_texts)
    scores = [(_cosine(q_emb, e), i) for i, e in enumerate(cand_embs)]
    scores.sort(key=lambda x: x[0], reverse=True)

    top_docs = [cand_docs[i] for (s, i) in scores[:k_final]]
    rag_text = "\n\n".join([d.page_content[:1200] for d in top_docs])
    return rag_text

print("✅ LLM/임베딩/검색(+재랭크, 내결함성) - 준비 완료")


⚠️ DuckDuckGoSearchAPIWrapper 임포트 실패: Could not import duckduckgo-search python package. Please install it with `pip install -U duckduckgo-search`.
↻ duckduckgo-search 설치 시도 중...
✅ duckduckgo-search 설치 완료
✅ LLM/임베딩/검색(+재랭크, 내결함성) - 준비 완료


In [65]:
# part 2 (3) 
#==========================================================
# 📌 도메인 가드 (치매상담 범위 밖 → 차단)
#==========================================================

ALLOW_HINTS = [
    "치매", "경도인지장애", "알츠하이머", "인지", "기억", "낮은 기억력", "단어", "길을 잃",
    "MMSE", "MoCA", "신경심리검사", "뇌파", "보호자", "일상생활", "방향감각", "언어유창성",
    "건망", "증상", "상담", "의사", "전문의", "정서", "우울", "불안"
]
BLOCK_HINTS = [
    "콜라", "당분", "버거킹", "햄버거", "메뉴", "주식", "ETF", "게임", "여행",
    "세금", "부동산", "스포츠", "연예", "날씨", "환율", "가전", "쇼핑"
]

GUARD_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system",
         "당신은 입력이 '치매/인지장애 상담' 범위에 해당하는지 판별하는 필터입니다. "
         "예/아니오만 한국어로 답하세요. 예시: '예' 또는 '아니오'"),
        ("human",
         "입력:\n{user_text}\n\n"
         "판별 기준: 치매, 경도인지장애, 인지/기억/언어/방향감각 문제, 가족/보호자 상담 등과 밀접하면 '예', "
         "그 외(음료 당분, 패스트푸드 신메뉴 등)면 '아니오'.")
    ]
)

guard_llm   = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.0, max_tokens=3, api_key=OPENAI_API_KEY)
guard_chain = GUARD_PROMPT | guard_llm | StrOutputParser()

def is_on_topic(text: str) -> bool:
    if any(b in text for b in BLOCK_HINTS) and not any(a in text for a in ALLOW_HINTS):
        return False
    if any(a in text for a in ALLOW_HINTS):
        return True
    try:
        verdict = guard_chain.invoke({"user_text": text}).strip()
        return verdict.startswith("예")
    except Exception:
        return True

print("✅ 도메인 가드 - 준비 완료")


✅ 도메인 가드 - 준비 완료


In [66]:
# part 3 (1) 
#==========================================================
# 📌 검색질의 생성기 (STT → 관련 검색어 2~4개)
#==========================================================
QUERY_PROMPT = ChatPromptTemplate.from_messages(
    [
      ("system",
       "너는 의료 상담 보조 검색어 생성기이다. 입력 STT에서 핵심 증상/키워드를 뽑아 "
       "치매/경도인지장애와 관련된 한국어 검색질의 2~4개를 JSON 배열로만 출력하라."),
      ("human", "{transcript}")
    ]
)
query_llm   = ChatOpenAI(model=CHAT_MODEL_NAME, temperature=0.1, max_tokens=120, api_key=OPENAI_API_KEY)
query_chain = QUERY_PROMPT | query_llm | StrOutputParser()

def make_search_queries(transcript: str) -> List[str]:
    try:
        raw = query_chain.invoke({"transcript": transcript})
        qs = json.loads(raw)
        qs = [q for q in qs if isinstance(q, str)]
        base = ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]
        return (qs + base)[:4]
    except Exception:
        return ["치매 초기 증상", "경도인지장애 언어 유창성", "일상 안전 가족 교육", "단기 기억력 저하 원인"]

print("✅ 검색질의 생성기 - 준비 완료")


✅ 검색질의 생성기 - 준비 완료


In [67]:
# part 3 (2) 
#==========================================================
# 📌 요약 체인 (템플릿 강제) — 재랭크 기반 RAG 문맥 사용
#==========================================================
summary_chain = SUMMARISE_PROMPT | llm | StrOutputParser()

def summarise_transcript(
    transcript: str,
    guide_question_index: int = 3,  # 기본: "최근에 불편했던 점이나 걱정되는 점이 있나요?"
) -> str:
    if not is_on_topic(transcript):
        return OFF_TOPIC_MESSAGE

    guide_question = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]

    # 1) 질의 생성
    queries = make_search_queries(transcript)

    # 2) RAG 문맥 (Rerank-Then-Summarise)
    rag_context = build_rag_context_rerank(transcript, queries)

    # 3) 요약 생성
    out = summary_chain.invoke({
        "transcript": transcript.strip(),
        "rag_context": rag_context.strip() if rag_context else "(문맥 없음)",
        "guide_question": guide_question,         # ← 프롬프트 자리표시자에 주입
        "summary_template": SUMMARY_TEMPLATE,     # ← 프롬프트 자리표시자에 주입
    })
    return out

def summarise_from_txt(path: str, guide_question_index: int = 3, encoding="utf-8") -> str:
    with open(path, "r", encoding=encoding) as f:
        text = f.read()
    return summarise_transcript(text, guide_question_index=guide_question_index)

print("✅ 요약 체인 - 준비 완료")


✅ 요약 체인 - 준비 완료


In [68]:
# part 4 (demo)
#==========================================================
# 📌 데모 실행 (직접 텍스트/파일로 테스트)
#==========================================================
DEMO_STT = """네, 요즘 물건을 어디에 두었는지 자꾸 잊어버려서 힘듭니다. 
안경이나 휴대폰을 두고도 찾지를 못해서 가족들한테 자주 물어보게 돼요. 
또 밖에 나갔다가 길을 잃을까 봐 혼자 외출하는 것도 무섭습니다. 
집에서는 전화 통화를 하고 나면 금방 무슨 이야기를 했는지 기억이 안 나서 답답하고요. 
제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다. 
대화할 때 단어가 잘 떠오르지 않아서 사람들이 이상하게 생각할까 봐 창피하기도 합니다."""

print("====== 요약 결과 (샘플) ======")
print(summarise_transcript(DEMO_STT, guide_question_index=3))


====== 요약 결과 (샘플) ======


c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


<질문> 이부분은 고정4개중 1개 - 챗봇이 처음 제시한 가이드 질문  
“최근에 불편했던 점이나 걱정되는 점이 있나요?”

<STT:시나리오:전체상담내용>  
“네, 요즘 물건을 어디에 두었는지 자꾸 잊어버려서 힘듭니다. 안경이나 휴대폰을 두고도 찾지를 못해서 가족들한테 자주 물어보게 돼요. 또 밖에 나갔다가 길을 잃을까 봐 혼자 외출하는 것도 무섭습니다. 집에서는 전화 통화를 하고 나면 금방 무슨 이야기를 했는지 기억이 안 나서 답답하고요. 제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다. 대화할 때 단어가 잘 떠오르지 않아서 사람들이 이상하게 생각할까 봐 창피하기도 합니다.”

<요약>  
1. **주 증상**  
- 물건 잃어버림  
- 외출 시 길 잃을까 두려움  
- 전화 통화 후 기억 상실  
- 단어 떠오르지 않음  

2. **상담내용**  
- 물건을 두고 잊어버려 가족에게 자주 물어봄  
- 혼자 외출하는 것이 두려움  
- 전화 통화 후 대화 내용 기억하지 못함  
- 자신의 상태가 나빠져 가족에게 짐이 될까 걱정  
- 대화 중 단어가 잘 떠오르지 않아 수치심 느낌  

3. **심리상태**  
- 불안  
- 수치심  
- 답답함  
- 걱정  

4. **AI 해석**  
- 현재의 증상은 경도인지장애나 초기 치매와 관련이 있을 수 있음.  
- 증상이 점진적으로 악화될 가능성이 있음.  

5. **주의사항**  
- 일상에서의 안전을 위해 외출 시 동행을 권장함.  
- 정서적 지지를 위해 가족과의 소통을 강화할 필요가 있음.  
- 전문적인 검진을 통해 정확한 상태를 평가받는 것이 권장됨.  


In [69]:
# part 5 (optional cli)
#==========================================================
# 📌 간단 대화 루프 (오프토픽 차단 + 요약)
#==========================================================
# 사용법:
#  - 첫 발화는 4개 고정 질문 중 하나를 출력
#  - 이후 사용자의 긴 응답(텍스트)을 받아 요약 템플릿으로 정리

def start_chat(guide_question_index: int = 3):
    q = GUIDE_QUESTIONS[max(0, min(3, int(guide_question_index)))]
    print(f"[상담 챗봇] {q}")
    print("[안내] 긴 답변을 입력하세요. '/quit' 입력 시 종료.")
    while True:
        user = input("\n[사용자] ").strip()
        if user.lower() in {"/quit", "quit", "exit"}:
            print("종료합니다.")
            break
        result = summarise_transcript(user, guide_question_index=guide_question_index)
        print("\n[요약]\n")
        print(result)

# start_chat(guide_question_index=3)  # ← 필요 시 주석 해제하여 실행
print("✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행")


✅ (선택) CLI 루프 준비됨 — 필요 시 start_chat() 실행


In [70]:
# part 6
#==========================================================
# 📌 간단 대화 루프 (오프토픽 차단 + 요약)
#==========================================================
print(summarise_from_txt("./script_1.txt", guide_question_index=3))


c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\smhrd\Anaconda3\envs\LLM\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


<질문> 이부분은 고정4개중 1개 - 챗봇이 처음 제시한 가이드 질문  
“최근에 불편했던 점이나 걱정되는 점이 있나요?”

<STT:시나리오:전체상담내용>  
“요즘 물건을 어디에 두었는지 자꾸 잊어버려서 힘듭니다. 안경이나 휴대폰을 두고도 찾지를 못해서 가족들한테 자주 물어보게 돼요. 또 밖에 나갔다가 길을 잃을까 봐 혼자 외출하는 것도 무섭습니다. 집에서는 전화 통화를 하고 나면 금방 무슨 이야기를 했는지 기억이 안 나서 답답하고요. 제 상태가 점점 더 나빠져서 가족들에게 짐이 될까 봐 가장 걱정입니다. 대화할 때 단어가 잘 떠오르지 않아서 사람들이 이상하게 생각할까 봐 창피하기도 합니다.”

<요약>  
1. **주 증상**  
- 물건을 잃어버림  
- 길을 잃을까 두려움  
- 전화 통화 후 기억 상실  
- 단어가 잘 떠오르지 않음  

2. **상담내용**  
- 물건을 두고 찾지 못해 가족에게 자주 물어봄  
- 혼자 외출하는 것이 두려움  
- 전화 통화 후 대화 내용이 기억나지 않음  
- 상태가 나빠져 가족에게 짐이 될까 걱정  
- 대화 중 단어가 떠오르지 않아 창피함  

3. **심리상태**  
- 불안  
- 답답함  
- 걱정  
- 수치심  

4. **AI 해석**  
- 현재의 증상은 경도인지장애 또는 초기 치매와 관련이 있을 수 있음.  
- 전문가와 상담을 통해 정확한 평가가 필요할 것으로 보임.  

5. **주의사항**  
- 일상생활에서 안전을 위해 혼자 외출하지 않도록 주의.  
- 가족과의 대화를 통해 정서적 지지를 받는 것이 중요.  
- 증상이 지속되거나 악화될 경우 전문의와 상담하여 검진을 받는 것이 권장됨.
